In [0]:
%run "/Workspace/Formula1Project/includes/configuration"

In [0]:
%run "/Workspace/Formula1Project/includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")
print(v_data_source)

In [0]:
dbutils.widgets.text("p_file_date", "2020-01-01")
v_file_date = dbutils.widgets.get("p_file_date")
print(v_file_date)

In [0]:
accounts_key = dbutils.secrets.get(scope="formala1-scope", key="account-key")
spark.conf.set("fs.azure.account.key.formala1.dfs.core.windows.net", accounts_key)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
pitstops_schema = StructType([StructField("raceId", IntegerType(), False), 
                              StructField("driverId", IntegerType(), False), 
                              StructField("stop", IntegerType(), False), 
                              StructField("lap", IntegerType(), False), 
                              StructField("time", StringType(), False),
                              StructField("duration", StringType(), False),
                              StructField("milliseconds", IntegerType(), False)])

In [0]:
pitstops_df = spark.read.json(f"{raw_path}/{v_file_date}/pit_stops.json",schema=pitstops_schema, multiLine=True)

In [0]:
display(pitstops_df)

In [0]:
from pyspark.sql.functions import current_timestamp, lit
pitstops_df = pitstops_df.withColumnRenamed("driverId", "driver_id").withColumnRenamed("raceId", "race_id").withColumn("ingestion_date", current_timestamp()).withColumn("file_date", lit(v_file_date))

In [0]:
merge_condition = "tgt.race_id = src.race_id and tgt.driver_id = src.driver_id and tgt.stop = src.stop"
merge_data(pitstops_df, processed_path, "f1_processed", "pit_stops", "race_id", merge_condition)

In [0]:
#pitstops_df.write.mode("append").format("parquet").saveAsTable("f1_processed.pitstops")